In [18]:
import os
from PIL import Image, ImageSequence, ImageDraw
import random
import os
import math

In [22]:
# Load two GIFs and extract their first frames
gif1_path = "../gif/coolstar.gif"
gif2_path = "../gif/dollarspindownd.gif"
gif1 = Image.open(gif1_path)
gif2 = Image.open(gif2_path)
frame1 = gif1.copy().convert("RGBA")  # First frame of GIF 1
frame2 = gif2.copy().convert("RGBA")  # First frame of GIF 2
gif1_width, gif1_height = frame1.size
gif2_width, gif2_height = frame2.size

In [24]:
# Set up the output canvas (800x800)
canvas_size = 800
num_frames = 30
output_dir = "../gif/interpolated_dual_frames"
os.makedirs(output_dir, exist_ok=True)

# Center positions for both GIFs
center_x1, center_y1 = canvas_size // 4, canvas_size // 2  # Left-side center
center_x2, center_y2 = 3 * canvas_size // 4, canvas_size // 2  # Right-side center

# Function to generate final random positions for a GIF
def generate_random_positions(frame, center_x, center_y, spread_factor):
    pixels = list(frame.getdata())
    positions = []
    width, height = frame.size
    for y in range(height):
        for x in range(width):
            pixel_index = y * width + x
            pixel_color = pixels[pixel_index]

            # Calculate random final position
            spread_x = random.randint(-spread_factor, spread_factor)
            spread_y = random.randint(-spread_factor, spread_factor)
            final_x = center_x + x - width // 2 + spread_x
            final_y = center_y + y - height // 2 + spread_y

            # Store the original, final positions, and color
            if 0 <= final_x < canvas_size and 0 <= final_y < canvas_size:
                positions.append(((center_x + x - width // 2, center_y + y - height // 2), (final_x, final_y), pixel_color))
    return positions

# Generate final positions for both GIFs
spread_factor = canvas_size // 2  # Full canvas spread
final_positions1 = generate_random_positions(frame1, center_x1, center_y1, spread_factor)
final_positions2 = generate_random_positions(frame2, center_x2, center_y2, spread_factor)

def adjusted_logarithmic_interpolation(t):
    """
    Logarithmic interpolation with slower initial growth.
    Adjusts the base factor for smoother starting motion.
    """
    base = 100  # Higher base slows down the initial growth
    return math.log10(1 + (base - 1) * t) / math.log10(base)

# Interpolate positions for intermediate frames
for frame_index in range(num_frames + 1):
    t_linear = frame_index / num_frames  # Linear interpolation factor
    t_log = logarithmic_interpolation(t_linear)  # Convert to logarithmic factor
    frame = Image.new("RGBA", (canvas_size, canvas_size), (0, 0, 0, 255))
    draw = ImageDraw.Draw(frame)

    # Draw pixels for GIF 1
    for (orig_x, orig_y), (final_x, final_y), color in final_positions1:
        interp_x = int(orig_x + (final_x - orig_x) * t_log)
        interp_y = int(orig_y + (final_y - orig_y) * t_log)
        draw.point((interp_x, interp_y), fill=color)

    # Draw pixels for GIF 2 (overlapping GIF 1)
    for (orig_x, orig_y), (final_x, final_y), color in final_positions2:
        interp_x = int(orig_x + (final_x - orig_x) * t_log)
        interp_y = int(orig_y + (final_y - orig_y) * t_log)
        draw.point((interp_x, interp_y), fill=color)

    # Save the frame
    frame.save(f"{output_dir}/frame_{frame_index:03d}.png")

print(f"Interpolated frames for both GIFs with logarithmic motion saved in {output_dir}")



Interpolated frames for both GIFs with logarithmic motion saved in ../gif/interpolated_dual_frames
